In [1]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [2]:
%%bash
mkdir sagemaker_lorax/

cat <<EOF > sagemaker_lorax/sagemaker_entrypoint.sh
#!/bin/bash

if [[ -z "\${HF_MODEL_ID}" ]]; then
  echo "HF_MODEL_ID must be set"
  exit 1
fi
export MODEL_ID="\${HF_MODEL_ID}"

if [[ -n "\${HF_MODEL_REVISION}" ]]; then
  export REVISION="\${HF_MODEL_REVISION}"
fi

if [[ -n "\${SM_NUM_GPUS}" ]]; then
  export NUM_SHARD="\${SM_NUM_GPUS}"
fi

if [[ -n "\${HF_MODEL_QUANTIZE}" ]]; then
  export QUANTIZE="\${HF_MODEL_QUANTIZE}"
fi

if [[ -n "\${HF_MODEL_TRUST_REMOTE_CODE}" ]]; then
  export TRUST_REMOTE_CODE="\${HF_MODEL_TRUST_REMOTE_CODE}"
fi

if [[ -z "\${ADAPTER_BUCKET}" ]]; then
  echo "Warning: ADAPTER_BUCKET not set. Only able to load local or HuggingFace Hub models."
else
  export PREDIBASE_MODEL_BUCKET="\${ADAPTER_BUCKET}"
fi

lorax-launcher --port 8080
EOF

mkdir: sagemaker_lorax/: File exists


In [3]:
%%bash
cat <<EOF > sagemaker_lorax/Dockerfile
FROM ghcr.io/predibase/lorax:latest

COPY sagemaker_entrypoint.sh entrypoint.sh
RUN chmod +x entrypoint.sh

ENTRYPOINT ["./entrypoint.sh"]
EOF

In [10]:
!echo $PATH

/Users/ravi.tej/anaconda3/envs/recoenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin


In [11]:
import os

In [15]:
os.environ['PATH'] = '/opt/homebrew/opt/mongodb-community@5.0/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Users/ravi.tej/anaconda3/envs/recoenv/bin:/Users/ravi.tej/anaconda3/condabin'

In [16]:
!echo $PATH

/opt/homebrew/opt/mongodb-community@5.0/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Users/ravi.tej/anaconda3/envs/recoenv/bin:/Users/ravi.tej/anaconda3/condabin


In [22]:
%%bash 
algorithm_name="lorax"  # name of your algorithm
tag="sagemaker"
region="ap-south-1"

account=$(aws sts get-caller-identity --query Account --output text)

image_uri="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" --region $region > /dev/null
fi

cd sagemaker_lorax/ && docker build -t ${algorithm_name}:${tag} .

# Authenticate Docker to an Amazon ECR registry
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com

# Tag the image
docker tag ${algorithm_name}:${tag} ${image_uri}

# Push the image to the repository
docker push ${image_uri}

# Save image name to tmp file to use when deploying endpoint
echo $image_uri > /tmp/image_uri

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load .dockerignore
#1 transferring context: 2B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile
#2 transferring dockerfile: 243B done
#2 DONE 0.0s

#3 [internal] load metadata for ghcr.io/predibase/lorax:sha256-e1101f14171b056348f9e91d9975408158687d9cee84342c4b643605430a91ff
#3 ERROR: encountered unknown type application/vnd.amazon.soci.index.v1+json; children may not be fetched
------
 > [internal] load metadata for ghcr.io/predibase/lorax:sha256-e1101f14171b056348f9e91d9975408158687d9cee84342c4b643605430a91ff:
------
Dockerfile:1
--------------------
   1 | >>> FROM ghcr.io/predibase/lorax:sha256-e1101f14171b056348f9e91d9975408158687d9cee84342c4b643605430a91ff
   2 |     
   3 |     COPY sagemaker_entrypoint.sh entrypoint.sh
--------------------
ERROR: failed to solve: ghcr.io/predibase/lorax:sha256-e1101f14171b056348f9e91d9975408158687d9cee84342c4b643605430a91ff: encountered unknown ty

Login Succeeded


Error response from daemon: No such image: lorax:sagemaker


The push refers to repository [005418323977.dkr.ecr.ap-south-1.amazonaws.com/lorax]


An image does not exist locally with the tag: 005418323977.dkr.ecr.ap-south-1.amazonaws.com/lorax
